##############################################################################
##############################################################################
### **Atelier "Comprendre et développer des CNN pour le traitement d'images"**
##############################################################################
##############################################################################
<br>
<br>
<br>
<br>

#### **Partie 2 : Développement d'un CNN de classification d'images satellitaires from scratch** 



Dans cette deuxième partie, nous allons développer un modèle CNN "from scratch", afin de classifier correctement des images satellitaires de 4 types : désert, foret, océan et nuageux
<br>
<br>
<br>
<br>

############################################################






<br>
<br>

#### **Import des librairies utiles pour l'atelier**

L'atelier se base sur **Pytorch**, un framework qui a énormément gagné en popularité ces dernières années. 

**TorchVision** est lui un package spécialement tourné vers la Computer Vision, et permet l'accès facile à des datasets publics, de créer aisement ses propres datasets. Il contient aussi nativement les principaux modèlesde fondation Open-Source, comme nous le verrons plus tard.
<br>

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt

#### **Vérification de la présence de capacité de calcul GPU sur l'ordinateur**

In [ ]:
# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")


<br>

#### **Etape 1 : chargement du Dataset et transformation des données pour qu'elles soient utilisables par un modèle CNN**





<br>



Nous allons maintenant transformer les images pour qu'elles aient : 
- La taille requise pour être entrée dans le modèle : 128 pixels par 128 pixels
- Un format qui soit un tensor Pytorch, afin d'être intégré au Pipeline Pytorch


Nous allons utiliser l'utilitaire **Transforms de TorchVision** qui va automatiser le pré-traitement des éléments du dataset
<br>

#### **Chargement du Dataset avec l'utilitaire datasets de TorchVision**

TorchVision permet de manipuler de manière optimale les datasets, en leur appliquant des pipelines de transformation en optimisant le temps de calcul

La seule obligation est de fournir un répertoire où les images doivent être mises dans des dossiers contenant le nom des labels. Ces noms de dossier deviendront automatiquement les classes d'apprentissage

<br>




In [ ]:
# Paths
data_dir = "C:/Code_ML/Code/Atelier CNN/data/"
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Redimensionner les images
    transforms.ToTensor(),          # Convertir en tenseur
    
])
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
print(f"Total Images: {len(full_dataset)}")
print(f"Classes: {full_dataset.classes}")

<br>

Nous n'avons pas normalisé nos données, nous allons donc calculer les propriétés du dataset et appliquer ces valeurs à l'opération de normalisation

<br>

Ce qu'il faut savoir :

les images dans PyTorch sont stockées sous la forme d’un tenseur 4D : 
    images.shape = (B,C,H,W)

où :
- B : batch size
- C : nombre de canaux
- H : hauteur de l'image
- W : largeur de l'image

<br>

Si l'on veut par exemple calculer la moyenne sur une image, elle se mesure par canal, R, G et B par exemple. Cette moyenne est donc un tenseur de taille C, elle est calculée par l'opération suivante : images.mean([0,2,3]), soit la moyenne sur tout le batch, calculé sur tous les pixels

<br>

In [ ]:



full_dataset_loader = DataLoader(full_dataset, batch_size=32, shuffle=False, num_workers=4)

# Fonction pour calculer la moyenne et l'écart-type
def compute_mean_std(full_dataset_loader):
    mean = torch.zeros(3)
    std = torch.zeros(3)
    total_images = 0

    for images, _ in full_dataset_loader:
        batch_size, channels, height, width = images.shape
        total_images += batch_size
        #################################################################################
        # Calculez la somme des moyennes et des variances pour chaque canal
        #################################################################################
         
        mean += 
        std += 

    mean /= 
    std /= 

    return mean, std

# Calcul des statistiques
mean, std = compute_mean_std(full_dataset_loader)

print(f"Moyenne des canaux RGB: {mean}")
print(f"Écart-type des canaux RGB: {std}")



 # Nouvelle transformation avec normalisation
full_dataset.transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Redimensionner les images
    transforms.ToTensor(),
    transforms.Normalize(mean=mean.tolist(), std=std.tolist())
 ])




<br>

#### **Split du dataset**

Selon les bonnes pratiques du ML, nous allons spliter notre dataset en jeu de Train, Validation et Test. Nous allons ensuite les convertir en Loaders, qui permettront une manipulation optimale ultérieurement.


Le paramètre "Batch size" détermine la taille des batchs servant à itérer pour chaque mise à jour des poids.


On pourrait passer 1 image à la fois ou à l'inverse, tout le dataset, mais ce n'est pas optimal. Passer des batchs de taille adaptée permet de gagner en efficacité d'entrainemenet et en stabilité des gradients

<br>

Les proportions retenues entre les jeux de train, validation et test peuvent varier, mais pour des datasets de taille moyenne, on choisit un ratio de type [70%,15%,15%]

<br>



In [ ]:
train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Train: {len(train_dataset)}, Validation: {len(val_dataset)}, Test: {len(test_dataset)}")


**Avant de créer le modèle, visualisations quelques échantillons aléatoires du dataset**

In [ ]:
classes = full_dataset.classes
def denormalize(image, mean, std):
    mean = torch.tensor(mean).view(3, 1, 1)  # Adapter la forme pour le broadcasting
    std = torch.tensor(std).view(3, 1, 1)
    return image * std + mean  # Correcte inversion de la normalisation
def show_images(loader):
    dataiter = iter(loader)
    images, labels = next(dataiter)
    fig, axes = plt.subplots(1, 5, figsize=(15, 5))
    for i in range(5):
        img = denormalize(images[i], mean, std)  # Dénormaliser l'image
        img = img.permute(1, 2, 0).clamp(0, 1)  # Convertir en format HWC et s'assurer des valeurs valides [0,1]

        axes[i].imshow(img.numpy())  
        axes[i].set_title(classes[labels[i]])
        axes[i].axis('off')
    plt.show()

print("Sample images from training data:")
show_images(train_loader)


<br>

#### **Etape 2 : construction de l'architecture du CNN**


Nous allons maintenant constituer l'architecture de notre CNN.


Pour cela, Pytorch propose la définition d'un classe CNN dans lequel nous aurons définir 2 parties :
<br>

------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------
- **self.conv_layers** : dans laquelle nous allons ajouter de manière séquentielle les différentes strates de la partie convolutive du modèle avec leurs paramètres
    -   Exemple : **nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1)**

                - couche de convolution avec 16 matrices de convolution de taille 3X3X3, l'image ayant 3 canaux. La couche aura donc en sortie 32 cartes de features issus des 32 matrices.

                - stride = 2: le noyau de convolution se déplace 2 pixel par 2 pixel, ce qui créé une diminution de la taille de l'image

                - padding = 1 : On rajoute 1 pixel en bord d'image pour que le passage du filtre ne réduise pas la taille de la sortie 

------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------          
- **self.fc_layers** : dans laquelle nous allons ajouter de manière séquentielle les différentes strates de la partie "fully connected" du modèle avec leurs paramètres. Généralement, on va aplatir les données en sortie de la partie conventionneles pour les transformer en vecteur unidimensionnel ("flatten") et ainsi pouvoir ajouter une couche de neurones totalement connectés, sur 1 ou plusieurs couches, jusqu'à la couche finale qui contient autant de neurons que le nombre de classes à prédire (4 dans notre cas)

<br>

La fonction forward est standard et indique l'ordre de propagation des données dans le modèle



In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            #################################################################################
            #Complétez la couche convolutive 1 pour y mettre : 
            #   - 32 matrices de convolution de taille 3X3X3
            #   - pas de diminution de taille de l'image
            #    - un padding classique de 1 pixel
            #################################################################################
         
            nn.Conv2d(  
            nn.ReLU(),
            #################################################################################
            #Complétez la couche de pooling pour avoir une réduction de la taille des cartes par un facteur 2
            #################################################################################
            nn.MaxPool2d( 

            #################################################################################
            #Sur le modèle de la première séquence de 3 éléments, 
            # reproduisez cette sequence avec cette fois ci 
            #  - 64 matrices de convolution de taille 3X3X3
            #  - une fonction d'activation ReLU
            # Une Pooling diminuant la taille de la carte d'un facteur 2
            #################################################################################
         
            #################################################################################
            #Sur le modèle de la première séquence de 3 éléments, 
            # reproduisez cette sequence avec cette fois ci 
            #  - 128 matrices de convolution de taille 3X3X3
            #  - une fonction d'activation ReLU
            # Une Pooling diminuant la taille de la carte d'un facteur 2
            #################################################################################
         


        )

        self.fc_layers = nn.Sequential(
            
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

**Instantiation du modèle**

In [ ]:
model = CNN(num_classes=len(classes))
model = model.to(device)
print(model)

<br>

#### **Etape 3 : entrainement**


Nous allons maintenant procéder à l'entrainement du modèle.


Nous commencons par définir la métrique liée à la fonction de cout (ici l'entropie croisée)

<br>



In [ ]:
criterion = nn.CrossEntropyLoss()



Nous définissons ensuite l'**optimizer**, le grand classique étant Adam.

Regardons le paramètre lr ou **learning rate**:
Ce paramètre détermine l'intensité de mise à jour des poids du modèle. Ceci signifie qu'un lr trop grand engendre de fortes variations et donc une forte instabilité, alors qu'un lr trop faible rendrait très lent l'apprentissage.

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)


Nous allons maintenant entrainer le modèle sur un certain nombre d'époques, ici définis à 5.
<br>

Lors de chaque époque, le modèle va s'entrainer sur tous les éléments du dataset de train, puis sera évalué sur le dataset de validation.

l'objet Model a 2 modes, le "training mode" enclenché par l'appel model.train() et le "validation mode, enclenché par l'appel model.eval().

<br>

2 élements clefs à comprendre :

- **lors de la première phase** :
    1. la "forward pass" se passe lorsque l'input est passée dans le modèle pour générer des outputs  :outputs = model(inputs)
    2. l'erreur est calculée : loss = criterion(outputs, labels)
    3. la retropropagation d'erreur suit : loss.backward()
    4. les poids du modèles sont mis à jour : optimizer.step() 
- **lors de la deuxième phase**, l'erreur sur le jeu de validation est calculé. Cette valeur sert exclusivement à observer du surapprentissage


<br>
<br>

In [ ]:
num_epochs = 5
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    # Training
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    val_accuracy = 100 * correct / total

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

**Affichage des valeurs de loss et d'accuracy**

In [ ]:
import matplotlib.pyplot as plt

# Plot validation losses
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs + 1), val_losses, marker='o', color='b', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Validation Loss over Epochs')
plt.legend()
plt.grid(True)
plt.show()



<br>

#### **Etape 4 : Evaluation du modèle sur les données de Test, données extérieures à l'apprentissage**

<br>

In [ ]:

model.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= len(test_loader)
test_accuracy = 100 * correct / total

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

<br>
#### **Etape 5 : à vous de jouer !**

Prenez le temps d'étudier l'effet de différents paramètres :
- l'ajout d'une data augmentation
- la variation du batch size
- la variation du learning rate
- l'ajout de couches de convolution

<br>

**Question bonus** : pouvez-vous retrouver, à partir des caractéristiques du CNN initial et de sa première couche fully connected (nn.Linear(128 * 16 * 16, 256)), la nécessité d'avoir des images de format 128x128x3 en entrée?

<br>